In [ ]:
import pickle, os
from pyecharts import options as opts
from pyecharts.charts import Kline, Line, Grid, Bar, Scatter,Page
import pandas as pd
import akshare as ak
from log import logger
from fetch_stock_data import get_data_from_all


# 读取temp_data/stock_data_20210701_20230328.pkl
with open('../temp_data/stock_data_20210701_20230328.pkl', 'rb') as f:
    stock_data = pickle.load(f)


In [ ]:
# 读取股票数据
# df = stock_data[('600519', '贵州茅台')]


def kline_compare_base(stock_df, axis_index=0):
    # 将数据转换为列表格式
    data = stock_df[['open', 'close', 'low', 'high']].values.tolist()
    dates = stock_df.index.tolist()  # date type:str, like '2021-07-01'

    # 绘制 K 线图
    kline = (
        Kline(init_opts=opts.InitOpts(
            bg_color="#000000")).add_xaxis(dates).add_yaxis(
                series_name="K线图",
                y_axis=data,
                xaxis_index=axis_index,
                yaxis_index=axis_index,
                itemstyle_opts=opts.ItemStyleOpts(
                    color="#ec0000",
                    color0="#00da3c",
                    border_color="#8A0000",
                    border_color0="#008F28",
                ),
            ).set_global_opts(
                title_opts=opts.TitleOpts(title="双品种组合"),
                xaxis_opts=opts.AxisOpts(
                    # type_='category',
                    is_scale=True,
                    splitline_opts=opts.SplitLineOpts(is_show=False)),
                yaxis_opts=opts.AxisOpts(
                    is_scale=True,
                    splitline_opts=opts.SplitLineOpts(is_show=False)),
                legend_opts=opts.LegendOpts(
                    is_show=True,
                    textstyle_opts=opts.TextStyleOpts(color='#DCDCDC'))).
        set_series_opts(
            # label_opts=opts.LabelOpts(is_show=False),  # 去掉line上的标签(数值)
            label_opts=opts.LabelOpts(is_show=False)))

    # 绘制均线图
    # 计算均线数据
    ma5 = stock_df['close'].rolling(window=5).mean().tolist()
    ma10 = stock_df['close'].rolling(window=10).mean().tolist()
    ma20 = stock_df['close'].rolling(window=20).mean().tolist()

    line = (
        Line().set_series_opts(
            label_opts=opts.LabelOpts(is_show=False),  # 去掉line上的标签(数值)
        ).add_xaxis(dates).add_yaxis("MA5", ma5,
                                     symbol='none')  # symbol='none'去掉line上的圆点
        .add_yaxis("MA10", ma10, symbol='none').add_yaxis("MA20",
                                                          ma20,
                                                          symbol='none'))

    # 将 K 线图和均线图组合在一起
    k_ma = kline.overlap(line)
    return k_ma


def stock_compare_vol(stock_df, axis_index=0):
    data = stock_df['vol'].astype(
        int).values.tolist()  # pyecharts的数据类型必须是int,如果是float,则会无法显示
    dates = stock_df.index.tolist()  # date type:str, like '2021-07-01'
    bar = (
        Bar().set_global_opts(
            xaxis_opts=opts.AxisOpts(
                # type_='category',
                is_scale=True,
                splitline_opts=opts.SplitLineOpts(is_show=False),
                axislabel_opts=opts.LabelOpts(is_show=False)),
            yaxis_opts=opts.AxisOpts(
                is_scale=True,
                splitline_opts=opts.SplitLineOpts(is_show=False)),
            legend_opts=opts.LegendOpts(
                is_show=False,
                textstyle_opts=opts.TextStyleOpts(color='#DCDCDC'))).
        add_xaxis(dates).add_yaxis(
            series_name="成交量",
            y_axis=data,
            xaxis_index=axis_index,
            yaxis_index=axis_index,
            itemstyle_opts=opts.ItemStyleOpts(
                color="#ec0000",
                # color0="#00da3c",
                # border_color="#8A0000",
                # border_color0="#008F28",
            ),
        ).set_series_opts(label_opts=opts.LabelOpts(is_show=False)))

    # 绘制均线图
    # 计算均线数据
    # ma5 = stock_df['close'].rolling(window=5).mean().tolist()
    # ma10 = stock_df['close'].rolling(window=10).mean().tolist()
    # ma20 = stock_df['close'].rolling(window=20).mean().tolist()

    # line = (
    #     Line()
    #     .set_series_opts(
    #         label_opts=opts.LabelOpts(is_show=False),  # 去掉line上的标签(数值)
    #     )
    #     .add_xaxis(dates)
    #     .add_yaxis("MA5", ma5, symbol='none')  # symbol='none'去掉line上的圆点
    #     .add_yaxis("MA10", ma10, symbol='none')
    #     .add_yaxis("MA20", ma20, symbol='none')
    # )

    # 将 K 线图和均线图组合在一起
    # k_ma  = bar.overlap(line)
    return bar

# def stock_compare_macd(stock_df, axis_index=0):
#     # 计算MACD
#     # 计算EMA
#     def EMA(data, timeperiod=12):
#         return data.ewm(span=timeperiod, adjust=False).mean()

#     # 计算DIFF
#     def DIFF(data, timeperiod=12, timeperiod2=26):
#         return EMA(data, timeperiod) - EMA(data, timeperiod2)

#     # 计算DEA
#     def DEA(data, timeperiod=9):
#         return EMA(DIFF(data), timeperiod)

#     # 计算MACD
#     def MACD(data, timeperiod=12, timeperiod2=26, timeperiod3=9):
#         return 2 * (DIFF(data, timeperiod, timeperiod2) - DEA(data, timeperiod3))

#     # 计算MACD
#     stock_df['diff'] = DIFF(stock_df['close'])
#     stock_df['dea'] = DEA(stock_df['close'])
#     stock_df['macd'] = MACD(stock_df['close'])

#     # 绘制MACD
#     dates = stock_df.index.tolist()  # date type:str, like '2021-07-01'
#     macd = (
#         Bar().set_global_opts(
#             xaxis_opts=opts.AxisOpts(
#                 # type_='category',
#                 is_scale=True,
#                 splitline_opts=opts.SplitLineOpts(is_show=False),
#                 axislabel_opts=opts.LabelOpts(is_show=False)),
#             yaxis_opts=opts.AxisOpts(
#                 is_scale=True,
#                 splitline_opts=opts.SplitLineOpts(is_show=False)),
#             legend_opts=opts.LegendOpts(
#                 is_show=False,
#                 textstyle_opts=opts.TextStyleOpts(color='#DCDCDC'))).
#         add_xaxis(dates).add_yaxis(
#             series_name="MACD",
#             y_axis=stock_df['macd'].values.tolist(),
#             xaxis_index=axis_index,
#             yaxis_index=axis_index,
#             itemstyle_opts=opts.ItemStyleOpts(
#                 color="#ec0000",
#                 # color0="#00da3c",
#                 # border_color="#8A0000",
#                 # border_color0="#008F28",
#             ),
#         ).set_series_opts(label_opts=opts.LabelOpts(is_show=False)))

#     # 绘制DIFF
#     diff = (
#         Line().set_series_opts(

def line_base(start_value) -> Line:
    stock_a_lg_indicator_df = ak.stock_a_lg_indicator(symbol="600519")
    stock_a_lg_indicator_df = stock_a_lg_indicator_df[stock_a_lg_indicator_df['trade_date'].apply(lambda d: pd.Timestamp(d)) >= pd.Timestamp(start_value)]

    line = (
        Line()
        .add_xaxis(xaxis_data=stock_a_lg_indicator_df.trade_date.tolist())
            .add_yaxis(yaxis_index=0, series_name="pe", y_axis=stock_a_lg_indicator_df.pe.rolling(window=5).mean().tolist())
        .set_global_opts(
        )
    )
    return line

In [ ]:
def kline_compare(df_top,df_bottom,start_value,end_value):

    kline_top = kline_compare_base(df_top,axis_index=0)
    kline_bottom = kline_compare_base(df_bottom,axis_index=1)
    kline_top_vol = stock_compare_vol(df_top,axis_index=2)
    kline_top_bottom = stock_compare_vol(df_bottom,axis_index=3)
    line_1 = line_base(start_value)

    kline_top.set_global_opts(datazoom_opts=[
        opts.DataZoomOpts(is_show=False,start_value=start_value, end_value=end_value, range_start=None,range_end=None, xaxis_index=[0,0], pos_top="97%",pos_bottom="1%"),
        opts.DataZoomOpts(is_show=True,start_value=start_value, end_value=end_value, range_start=None,range_end=None, xaxis_index=[0,1], pos_top="97%", pos_bottom="1%"),
        opts.DataZoomOpts(is_show=False,start_value=start_value, end_value=end_value, range_start=None,range_end=None, xaxis_index=[0,2], pos_top="97%", pos_bottom="1%"),
        opts.DataZoomOpts(is_show=False,start_value=start_value, end_value=end_value, range_start=None,range_end=None, xaxis_index=[0,3], pos_top="97%", pos_bottom="1%"),
        ],
        # xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(is_show=False)),
    )

    grid = Grid(init_opts=opts.InitOpts(width="1600px", height="800px"))

    grid.add(kline_top, grid_opts=opts.GridOpts(pos_top="0%", pos_bottom="60%"))
    grid.add(kline_top_vol, grid_opts=opts.GridOpts(pos_top="40%", pos_bottom="50%"))
    grid.add(kline_bottom, grid_opts=opts.GridOpts(pos_top="50%", pos_bottom="10%"))
    grid.add(kline_top_bottom, grid_opts=opts.GridOpts(pos_top="90%", pos_bottom="0%"))
    grid.add(line_1, grid_opts=opts.GridOpts(pos_top="0%", pos_bottom="60%"))

    return grid


df_top=stock_data[('600519', '贵州茅台')]
df_top=stock_data[('688766', '普冉股份')]
# df_top=get_data_from_all('600519',stock_data)
df_bottom = stock_data[('zs.000001', '上证指数')]
gg = kline_compare(df_top, df_bottom, '2021-08-06', '2022-03-29')
kline_compare(df_top,df_bottom,'2021-08-06','2022-03-29').render_notebook()

In [ ]:
x_data = ['Apple', 'Huawei', 'Xiaomi', 'Oppo', 'Vivo', 'Meizu']
y_data = [123, 153, 89, 0, 98, 23]

scatter = (Scatter().add_xaxis(x_data).add_yaxis('', y_data))

page = Page(layout=Page.SimplePageLayout)
page.add(
    gg,
    scatter,
    # tab
)
page.render_notebook()

In [ ]:
# read temp_data/all_yjbb.pkl
with open('../temp_data/all_yjbb.pkl', 'rb') as f:
    all_yjbb = pickle.load(f)
all_yjbb

bb_data = all_yjbb[all_yjbb['股票代码']=='600519'].sort_values(by='最新公告日期',ascending=True)

# read temp_data/all_data.pkl
with open('../temp_data/stock_data_20210701_20230328.pkl', 'rb') as f:
    all_data = pickle.load(f)
k_data = all_data[('600519', '贵州茅台')]

k_data.reset_index(inplace=True)
logger.debug(len(bb_data))
bb_data.drop_duplicates(subset=['最新公告日期'], keep='first', inplace=True)
logger.debug(len(bb_data))

k_data['date'] = pd.to_datetime(k_data['date'])
bb_data['最新公告日期'] = pd.to_datetime(bb_data['最新公告日期'])

In [ ]:
merged_df = pd.merge_asof(bb_data,k_data, left_on='最新公告日期', right_on='date', direction='backward')
# 清晰merged_df中date1为None 的数据
merged_df = merged_df[merged_df['date'].notnull()]
bb_df = k_data.merge(
    merged_df.iloc[:, 1:17], how='left', left_on='date', right_on='date')
bb_df[bb_df['最新公告日期'].notnull()].iloc[:, 11:]


In [ ]:
import pyecharts.options as opts
from pyecharts.charts import Bar, Line
import numpy as np

"""
Gallery 使用 pyecharts 1.0.0
参考地址: https://echarts.apache.org/examples/editor.html?c=multiple-y-axis

目前无法实现的功能:

1、暂无
"""

colors = ["#5793f3", "#d14a61", "#675bba"]
x_data = bb_df['date'].tolist()
legend_list = ["蒸发量", "降水量", "平均温度"]
evaporation_capacity = np.nan_to_num(bb_df['净利润-净利润']).astype(int).tolist()
rainfall_capacity = np.nan_to_num(bb_df['营业收入-季度环比增长']).astype(int).tolist()
average_temperature = np.nan_to_num(bb_df['净利润-同比增长']).astype(int).tolist()

bar = (
    Bar(init_opts=opts.InitOpts(width="1260px", height="720px"))
    .add_xaxis(xaxis_data=x_data)
    .add_yaxis(
        series_name="蒸发量", y_axis=evaporation_capacity, yaxis_index=0, color=colors[1]
    )
    .add_yaxis(
        series_name="降水量", y_axis=rainfall_capacity, yaxis_index=1, color=colors[0]
    )
    .extend_axis(
        yaxis=opts.AxisOpts(
            name="蒸发量",
            type_="value",
            min_=0,
            max_=250,
            position="right",
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(color=colors[1])
            ),
            axislabel_opts=opts.LabelOpts(formatter="{value} ml"),
        )
    )
    .extend_axis(
        yaxis=opts.AxisOpts(
            type_="value",
            name="温度",
            min_=0,
            max_=25,
            position="left",
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(color=colors[2])
            ),
            axislabel_opts=opts.LabelOpts(formatter="{value} °C"),
            splitline_opts=opts.SplitLineOpts(
                is_show=True, linestyle_opts=opts.LineStyleOpts(opacity=1)
            ),
        )
    )
    .set_global_opts(
        yaxis_opts=opts.AxisOpts(
            type_="value",
            name="降水量",
            min_=0,
            max_=250,
            position="right",
            offset=80,
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(color=colors[0])
            ),
            axislabel_opts=opts.LabelOpts(formatter="{value} ml"),
        ),
        tooltip_opts=opts.TooltipOpts(
            trigger="axis", axis_pointer_type="cross"),
    )
)

line = (
    Line()
    .add_xaxis(xaxis_data=x_data)
    .add_yaxis(
        series_name="平均温度", y_axis=average_temperature, yaxis_index=2, color=colors[2]
    )
)

bar.overlap(line).render_notebook()


In [ ]:
import numpy as np
x_data = bb_df['date'].tolist()
# y_data = np.nan_to_num(bb_df['净资产收益率']).astype(int).tolist()
y_data = np.nan_to_num(bb_df['营业收入-季度环比增长']).astype(int).tolist()
c = (
    Bar()
    .add_xaxis(x_data)
    .add_yaxis("商家A", y_data, bar_min_width=10,bar_min_height=0)
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(
        is_scale=True,
        ),
        yaxis_opts=opts.AxisOpts(
            is_scale=True,
            # min_=0.001
        )
    )
    .render_notebook()
)
c

In [ ]:
import numpy as np
from pyecharts.charts import Bar

# 创建一个包含NaN值的NumPy数组
data = np.array([1, 2, np.nan, 4, 5])

# 将NaN值替换为0
data = np.nan_to_num(data)
data = data.astype(int).tolist()
print(data)
# data = [1,2,0,4,5]

# 创建一个柱状图，并添加数据
bar = Bar()
bar.add_xaxis(['A', 'B', 'C', 'D', 'E'])
bar.add_yaxis('数据', data)
bar.render_notebook()  


In [ ]:
# 虚假数据
x_data = ['Apple', 'Huawei', 'Xiaomi', 'Oppo', 'Vivo', 'Meizu']
y_data = [123, 153, 89, 0, 98, 23]

scatter = (Scatter().add_xaxis(x_data).add_yaxis('', y_data))

scatter.render_notebook()

In [ ]:
from pyecharts import options as opts
from pyecharts.charts import Bar, Grid, Line, Pie, Tab
from pyecharts.faker import Faker


def bar_datazoom_slider() -> Bar:
    c = (Bar().add_xaxis(Faker.days_attrs).add_yaxis(
        "商家A", Faker.days_values).set_global_opts(
            title_opts=opts.TitleOpts(title="Bar-DataZoom（slider-水平）"),
            datazoom_opts=[opts.DataZoomOpts()],
        ))
    return c


def line_markpoint() -> Line:
    c = (Line().add_xaxis(Faker.choose()).add_yaxis(
        "商家A",
        Faker.values(),
        markpoint_opts=opts.MarkPointOpts(
            data=[opts.MarkPointItem(type_="min")]),
    ).add_yaxis(
        "商家B",
        Faker.values(),
        markpoint_opts=opts.MarkPointOpts(
            data=[opts.MarkPointItem(type_="max")]),
    ).set_global_opts(title_opts=opts.TitleOpts(title="Line-MarkPoint")))
    return c


def pie_rosetype() -> Pie:
    v = Faker.choose()
    c = (Pie().add(
        "",
        [list(z) for z in zip(v, Faker.values())],
        radius=["30%", "75%"],
        center=["25%", "50%"],
        rosetype="radius",
        label_opts=opts.LabelOpts(is_show=False),
    ).add(
        "",
        [list(z) for z in zip(v, Faker.values())],
        radius=["30%", "75%"],
        center=["75%", "50%"],
        rosetype="area",
    ).set_global_opts(title_opts=opts.TitleOpts(title="Pie-玫瑰图示例")))
    return c


def grid_mutil_yaxis() -> Grid:
    x_data = ["{}月".format(i) for i in range(1, 13)]
    bar = (Bar().add_xaxis(x_data).add_yaxis(
        "蒸发量",
        [2.0, 4.9, 7.0, 23.2, 25.6, 76.7, 135.6, 162.2, 32.6, 20.0, 6.4, 3.3],
        yaxis_index=0,
        color="#d14a61",
    ).add_yaxis(
        "降水量",
        [2.6, 5.9, 9.0, 26.4, 28.7, 70.7, 175.6, 182.2, 48.7, 18.8, 6.0, 2.3],
        yaxis_index=1,
        color="#5793f3",
    ).extend_axis(yaxis=opts.AxisOpts(
        name="蒸发量",
        type_="value",
        min_=0,
        max_=250,
        position="right",
        axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(
            color="#d14a61")),
        axislabel_opts=opts.LabelOpts(formatter="{value} ml"),
    )).extend_axis(yaxis=opts.AxisOpts(
        type_="value",
        name="温度",
        min_=0,
        max_=25,
        position="left",
        axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(
            color="#675bba")),
        axislabel_opts=opts.LabelOpts(formatter="{value} °C"),
        splitline_opts=opts.SplitLineOpts(
            is_show=True, linestyle_opts=opts.LineStyleOpts(opacity=1)),
    )).set_global_opts(
        yaxis_opts=opts.AxisOpts(
            name="降水量",
            min_=0,
            max_=250,
            position="right",
            offset=80,
            axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(
                color="#5793f3")),
            axislabel_opts=opts.LabelOpts(formatter="{value} ml"),
        ),
        title_opts=opts.TitleOpts(title="Grid-多 Y 轴示例"),
        tooltip_opts=opts.TooltipOpts(trigger="axis",
                                      axis_pointer_type="cross"),
    ))

    line = (Line().add_xaxis(x_data).add_yaxis(
        "平均温度",
        [2.0, 2.2, 3.3, 4.5, 6.3, 10.2, 20.3, 23.4, 23.0, 16.5, 12.0, 6.2],
        yaxis_index=2,
        color="#675bba",
        label_opts=opts.LabelOpts(is_show=False),
    ))

    bar.overlap(line)
    return Grid().add(bar,
                      opts.GridOpts(pos_left="5%", pos_right="20%"),
                      is_control_axis_index=True)


tab = Tab()
tab.add(bar_datazoom_slider(), "bar-example")
tab.add(line_markpoint(), "line-example")
tab.add(pie_rosetype(), "pie-example")
tab.add(grid_mutil_yaxis(), "grid-example")
tab.render_notebook()

In [2]:
import akshare as ak
import pandas as pd
from tools import tool_trade_date_hist
from fetch_stock_data import fetch_one_stock_hist_from_ak
from log import logger

start_date = '20140101'
end_date= '20140630'
symbol = '000001'
period = 'daily'

stock_info = (('000001','深圳银行'),period,start_date,end_date,'')
index_info = (('zs.000001','上证指数'),period,start_date,end_date)


stock_data = fetch_one_stock_hist_from_ak(stock_info)

#删除stock_data中的10到20行
stock_data = stock_data.drop(stock_data.index[0:20])
stock_data = stock_data.drop(stock_data.index[40:45])

index_data = fetch_one_stock_hist_from_ak(index_info)
trade_dates = tool_trade_date_hist(start_date,end_date)

stock_data['date'] = pd.to_datetime(stock_data['date'])
trade_dates['trade_date'] = pd.to_datetime(trade_dates['trade_date'])

ndf = trade_dates.merge(stock_data,how='left',left_on='trade_date',right_on='date')

#close：如为Nan，有前值使用前值填充，无前值使用后值填充
#date：如为Nan，使用suspension填充（停盘）
#open,high,low：如为Nan，使用close填充
#其它值：如为Nan，使用0填充

ndf['close'] = ndf['close'].fillna(method='ffill')
ndf['close'] = ndf['close'].fillna(method='bfill')
ndf['date'] = ndf['date'].fillna('suspension')
ndf['open'] = ndf['open'].fillna(ndf['close'])
ndf['high'] = ndf['high'].fillna(ndf['close'])
ndf['low'] = ndf['low'].fillna(ndf['close'])
ndf = ndf.fillna(0)

from chart_componets import kline
kline(ndf,date_name='trade_date').render_notebook()


In [2]:
from tools import tool_data_cleaning_by_trade_date_hist

df  = tool_data_cleaning_by_trade_date_hist(stock_data,'20140101','20140630')

kline(df, date_name='trade_date').render_notebook()


    trade_date                 date   open  close   high    low       vol  \
0   2014-01-02           suspension  11.40  11.40  11.40  11.40       0.0   
1   2014-01-03           suspension  11.40  11.40  11.40  11.40       0.0   
2   2014-01-06           suspension  11.40  11.40  11.40  11.40       0.0   
3   2014-01-07           suspension  11.40  11.40  11.40  11.40       0.0   
4   2014-01-08           suspension  11.40  11.40  11.40  11.40       0.0   
..         ...                  ...    ...    ...    ...    ...       ...   
114 2014-06-24  2014-06-24 00:00:00   9.78   9.80   9.85   9.76  284593.0   
115 2014-06-25  2014-06-25 00:00:00   9.80   9.77   9.81   9.64  271263.0   
116 2014-06-26  2014-06-26 00:00:00   9.71   9.82   9.83   9.71  342885.0   
117 2014-06-27  2014-06-27 00:00:00   9.79   9.82   9.87   9.71  367521.0   
118 2014-06-30  2014-06-30 00:00:00   9.80   9.91   9.97   9.79  447238.0   

          amount  swing  pct_chg  change  turn_over  
0            0.0   0.

In [4]:
a = tool_trade_date_hist('20140101','20140630')
a


,trade_date
5636,2014-01-02
5637,2014-01-03
5638,2014-01-06
5639,2014-01-07
5640,2014-01-08
...,...
5750,2014-06-24
5751,2014-06-25
5752,2014-06-26
5753,2014-06-27


In [ ]:
import akshare as ak

stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol="000001",
                                        period="daily",
                                        start_date="20170301",
                                        end_date='20210907',
                                        adjust="")
print(stock_zh_a_hist_df)

In [ ]:
import akshare as ak

def tool_trade_date_hist(strat_date,end_date):
    tool_trade_date_hist_sina_df = ak.tool_trade_date_hist_sina()
    # 截取tool_trade_date_hist_sina_df 1991-04-03到2023-03-30的数据
    tool_trade_date_hist_sina_df = tool_trade_date_hist_sina_df[tool_trade_date_hist_sina_df['trade_date'].apply(lambda x:pd.Timestamp(x))  >= pd.Timestamp(strat_date)]
    tool_trade_date_hist_sina_df = tool_trade_date_hist_sina_df[tool_trade_date_hist_sina_df['trade_date'].apply(lambda x:pd.Timestamp(x))  <= pd.Timestamp(end_date)]
    return tool_trade_date_hist_sina_df